In [30]:
import os
import io
import pandas as pd
import tensorflow as tf
import sys
sys.path.append("../../models/research")

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

def class_text_to_int(row_label):
    if row_label == 'conteiner':
        return 1
    else:
        None

def load_labels(arquivo):
    # Função está retornando apenas a primeira bbox
    labels = []
    with open(arquivo) as labels_in:
        line = labels_in.readline()
        c, xcen, ycen, w, h = map(float, line.split())
        x1 = xcen - w / 2
        x2 = xcen + w / 2
        y1 = ycen - h / 2
        y2 = ycen + h / 2
    return x1, x2, y1, y2

def create_tf_example(path, imageid):
    jpeg_filename = os.path.join(path, '{}.jpg'.format(imageid))
    txt_filename = os.path.join(path, '{}.txt'.format(imageid))
    with tf.io.gfile.GFile(os.path.join(path, '{}.jpg'.format(imageid)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = jpeg_filename.encode('utf8')
    image_format = b'jpg'
    xmin, xmax, ymin, ymax = load_labels(txt_filename)
    xmins = [xmin]
    xmaxs = [xmax]
    ymins = [ymin]
    ymaxs = [ymax]
    classes_text = [b'conteiner']
    classes = [class_text_to_int('conteiner')]

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [31]:
try:
    os.mkdir('../bases/tfrecord')
except FileExistsError as err:
    print(err)

[Errno 17] File exists: '../bases/tfrecord'


In [39]:
base_path = '/'.join(os.getcwd().split('/')[:-1])
train_image_dir = os.path.join(base_path, 'bases', 'baseline/')
output_dir = os.path.join(base_path, 'bases', 'tfrecord')
print(train_image_dir)
print(output_dir)

/home/ivan/PycharmProjects/ajna_bbox/bases/baseline/
/home/ivan/PycharmProjects/ajna_bbox/bases/tfrecord


In [40]:
image_ids = os.listdir(train_image_dir)
print(len(image_ids))

281


In [41]:
writer = tf.io.TFRecordWriter(os.path.join(output_dir, 'train.record'))
for image_id in image_ids[:260]:
    tf_example = create_tf_example(os.path.join(train_image_dir, image_id), image_id)
    writer.write(tf_example.SerializeToString())
writer.close()

In [42]:
writer = tf.io.TFRecordWriter(os.path.join(output_dir, 'test.record'))
for image_id in image_ids[260:]:
    tf_example = create_tf_example(os.path.join(train_image_dir, image_id), image_id)
    writer.write(tf_example.SerializeToString())
writer.close()